# Sprite Texture Pack Creation for Bloxd
The texture pack for Bloxd.io's Cube Warfare is single column of 266 16x16 pixel blocks.

This notebook shows one way to generate a texture pack file for use in the game. If no custom blocks are set, this will create all transparent blocks with a red border, and a block number indicated inside of the block face.

## Requirements
**pygame**  
Pygame is a free and open-source cross-platform library for the development of multimedia applications like video games using Python. It uses the Simple DirectMedia Layer library and several other popular libraries to abstract the most common functions, making writing these programs a more intuitive task.



In [16]:
# Import libraries
import pygame
from pathlib import Path

#Set a suitable font size
font_size = 16

# Define some colors constants for ease of use/convenience
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
TRANSPARENT = (0, 0, 0, 0)  # Transparent color

# Set this to false to override the block level setting
# this is useful when trying to identify blocks
show_number_override = False

# Define sprite dimensions
width = 16
height = 16
block_count = 266
total_height = block_count * height

# Create an empty dictionary of the custom blocks
custom_blocks = dict()

# Path and file to save newly created sprite:
saveto = Path("sprites/bloxd-tpack-wallhack.png")
saveto.parent.mkdir(parents=True, exist_ok=True)  # Create directories if needed


## Custom Blocks
This is where the blocks, identified by number (starting at 1) are customized. The general approach here, are there are likely some better ways, is to group similar block types which all receive the same properties. It was done this way with thought of eventually reading these values from a shared Google spreadsheet (not implemented).

Colors are defined by RGBA, with all values being between 0 and 255. The alpha is not a decimal value. It seems like the game ignores the alpha value if it is not 0 or 255. Partial transparency doesn't appear to work for blocks.

In [17]:
# Floors
floors = [2,4,19,29,31,81,130,]
for b in floors:
    custom_blocks[b] = {'fill_color': (55,58,62), 'border_width': 0, 'show_number':True, }

# Lower walls  
lower_walls = [35, 127,] 
for b in lower_walls:
    custom_blocks[b] = {'fill_color': (159,107,88), 'border_width': 0, 'show_number':True, }

# Boxes
boxes = [70,72,78,77,79,80,]
for b in boxes:
    custom_blocks[b] = {'fill_color': (73,59,91), 'border_width': 0, 'show_number':True, }

# Structural Walls
struct_walls = [15,126,]
for b in struct_walls:
    custom_blocks[b] = {'fill_color': TRANSPARENT, 'border_width': 2, 'border_color':(143,80,63), 'show_number':True, }

# Structural Walls2
struct_walls = [68, 69,71,88,95,105,]
for b in struct_walls:
    custom_blocks[b] = {'fill_color': TRANSPARENT, 'border_width': 2, 'border_color':(251,248,239), 'show_number':True, }

# Structural Walls 3
struct_walls3 = [27,]
for b in struct_walls3:
    custom_blocks[b] = {'fill_color': TRANSPARENT, 'border_width': 2, 'border_color':(128,124,124), 'show_number':True, }

# Tree trunks
trees = [10,]
for b in trees:
    custom_blocks[b] = {'fill_color': (97,60,32), 'border_width': 0, 'show_number':True, }

# Leaves
leaves = [98,100,]
for b in leaves:
    custom_blocks[b] = {'fill_color': (73,91,36), 'border_width': 0, 'show_number':True, }

## Create the Sprite
This is the bit that creates the file and saves it to disk.

In [18]:
# initialize pygame
pygame.init()

# The font is to be used on the block faces while developing/identifying blocks
font = pygame.font.SysFont(None, font_size)  # A suitable font and size

# Create the surface for the sprite
sprite_surface = pygame.Surface((width, total_height), pygame.SRCALPHA)  # Includes setting alpha flag
sprite_surface.set_colorkey(TRANSPARENT)  # Set transparent color

# Define default block set of properties
default_block = {
    "border_color": RED,
    "fill_color": TRANSPARENT,
    "border_width": 2,
    "show_number": True,
}

# Create a dictionary for each block 
blocks = {block_number: default_block.copy() for block_number in range(1, block_count + 1)} 

# Apply any customizations defined in the custom_blocks
for custom_key, custom_block_data in custom_blocks.items():
    try:
        blocks[custom_key].update(custom_block_data)
    except KeyError:
        print(f"Invalid custom key index of {custom_key}")

# Loop through each block for drawing
for block_number, block_data in blocks.items():

    border_color = block_data["border_color"]
    fill_color = block_data["fill_color"]
    border_width = block_data["border_width"]
    
    # Calculate block y position; the maths later is better if things are zero based,
    # so block_index represents the block position in those equations relying on geometry
    block_index = block_number - 1
    y_pos = block_index * height

    # Fill the center with the specified color (or transparent)
    fill_rect = (border_width, y_pos + border_width, width - 2 * border_width, height - 2 * border_width)
    pygame.draw.rect(sprite_surface, fill_color, fill_rect)

    # Draw the border with the specified color and width if the width has a value
    if bool(border_width):
        pygame.draw.rect(sprite_surface, border_color, (0, y_pos, width, height), border_width)

    if show_number_override and block_data["show_number"]:
        # Calculate text position (centered)
        text_surface = font.render(str(block_number), True, WHITE)  
        text_rect = text_surface.get_rect(center=(width // 2, block_index * height + height // 2))
    
        # Blit the text onto the sprite surface
        sprite_surface.blit(text_surface, text_rect)

#png_surface = sprite_surface.convert_alpha()
pygame.image.save_extended(sprite_surface, saveto)

# (Optional) Quit Pygame (if using it for rendering)
pygame.quit()

In [ ]:
# not necessary, but sometimes helpful in making sure things are being created as desired.
for key, block in blocks.items():
    print(key, block)